# Spark Lab 2: MLLib

In this lab we will explore the MLLib library for machine learning in Spark. The API of this library is very similar to Scikit Learn, and it plays quite nicely with Pandas.

This lab follows quite closely [this blog post](https://www.mapr.com/blog/churn-prediction-pyspark-using-mllib-and-ml-packages), so if you're lost you can go have  look there for guidance.

Let's start with the usual:
    - vagrant up
    - vagrant ssh

Now you should have access to Jupyter notebook here:

    http://10.211.55.101:18888/tree
    
The problem we will solve is the prediction of [_churn rate_](https://en.wikipedia.org/wiki/Churn_rate), which is a measure of how many customers are lost over a period of time. This is a very important business metric, in particular for large companies like Telecom companies.

We will use a dataset provided by [BigML](https://bigml.com/). The data has been copied to your VM, but can also be downloaded [here](https://bml-data.s3.amazonaws.com/churn-bigml-80.csv) and [here](https://bml-data.s3.amazonaws.com/churn-bigml-20.csv).

In [ ]:
# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import pandas as pd
pd.options.display.mpl_style = 'default'

Check that the SparkContext and sqlContext are available

In [ ]:
sc

In [ ]:
sqlContext

## Exercise 1.a: Load the data

Let's start by loading the data. Since the input is a CSV file we'll need to provide a parser.

- Use the sqlContext.read.load function to load the data
    - load the bigml-80 file to an RDD called CV_data
    - load the bigml-20 file to an RDD called final_test_data
    - cache CV_data to speed up things
    
Note that you can print the schema of the RDD if you want to


## Exercise 1.b: Quick look at the data

- use the `take` function to take the first 5 lines of the `CV_data` RDD and display them as Pandas dataframe
- correct the column types of the RDD
- use the `describe` function to have some summary statistics about the training data 

In [ ]:
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType

## Exercise 2: Sample inspection

Not all the features are numeric. `CV_data.dtypes` contains information on the type.

- select the features that are either `int` or `double`
- use the `sample` function to get a 10% sample of the training RDD
- Display a Pandas.scatter_matrix of the sampled data

## Exercise 3: Feature selection

Column selection on an RDD works differently than in Scikit Learn. For example if we want to drop 2 columns in Spark, we just apply the `.drop(column)` function 2 times.

- Drop the following columns:
    - State
    - Area Code
    - Total day charge
    - Total eve charge
    - Total night charge
    - Total intl charge
    
Also, we can apply a function to a column with the construct:

    .withColumn('column_name', function(CV_data['column_name']))
    
Use it to transform binary string labels to `1.0` or `0.0`. Treat these columns:

    - Churn
    - International plan
    - Voice mail plan

You may need these two imports:

```python
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import UserDefinedFunction
```

Also, use the `.cache` function to cache your pipeline results so far.

As before, take 5 lines and display them with Pandas

## Exercise 4: Train Decision Tree

Time has come to do our first model using MLLib. We will use a decision tree.

- [LabeledPoint](https://spark.apache.org/docs/0.8.1/api/mllib/org/apache/spark/mllib/regression/LabeledPoint.html) allows us to represent a data point with features and labels. Map it across the data using a function
- `.randomSplit` allows us to split the data in train/test sets. Do an 80/20 split
- Train a [DecisionTree](http://spark.apache.org/docs/latest/mllib-decision-tree.html) on the training data
- Display the trained model using `print model.toDebugString()`

You may need the following imports:

```python
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
```

## Exercise 5: Model valuation


The MulticlassMetrics module contains a lot of metrics functions.

- Evaluate the model on the test data using `.predict`
- Calculate the following metrics:
    - Precision of True 
    - Precision of False
    - Recall of True    
    - Recall of False   
    - F-1 Score         
    - Confusion Matrix

- Finally, display how many 

```python
from pyspark.mllib.evaluation import MulticlassMetrics
```

## Bonus: Cross Validation

The [original blog post mentioned above](https://www.mapr.com/blog/churn-prediction-pyspark-using-mllib-and-ml-packages) also contains code to implement cross validation. Try it and see if you understand how it's done.